In [1]:
import numpy as np
import pandas as pd
import os.path as path
import subprocess
from tqdm import tqdm
import glob

data_path = "../data/"
results_path = "../results/"
runs = pd.read_csv(data_path+'kernel_list.csv')
runtimes_path = results_path+"runtimes_temp.csv"
run_times = pd.DataFrame(columns=['path',"function","time",'blocks','matrix'])
if path.exists(runtimes_path):
    run_times = pd.read_csv(runtimes_path,low_memory=False)

Add the file path to the dataframe

In [2]:
runs['kernel_path'] = [data_path+"kernels/"+str(r["Repo"])+"/"+str(r["underdirectory"]) 
                        for _, r in runs.iterrows()]

In [3]:
runs['main_file'] = runs['kernel_path']+"/"+"main.cu"

Explore the dataset

In [4]:
runs.head()

,Repo,underdirectory,function,status,variables,kernel_path,main_file
0,1,0,euclidean_kernel,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/0,../data/kernels/1/0/main.cu
1,1,1,euclidean_kernel_same,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/1,../data/kernels/1/1/main.cu
2,1,2,maximum_kernel,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/2,../data/kernels/1/2/main.cu
3,1,3,maximum_kernel_same,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/3,../data/kernels/1/3/main.cu
4,1,4,manhattan_kernel,runs,"[('const float *', 'vg_a'), ('size_t', 'pitch_...",../data/kernels/1/4,../data/kernels/1/4/main.cu


In [5]:
print(len(runs))

20258


Try to compile some of the kernels. We follow the procedure in https://ntnuopen.ntnu.no/ntnu-xmlui/handle/11250/2981317. We only keep the code that is run on GPU.

In [6]:
compilation_command = "clang++-10 {} -pthread --cuda-gpu-arch=sm_75 -std=c++11 -fno-exceptions -stdlib=libstdc++ -S -emit-llvm -O3 --cuda-device-only -o {}"

num_kernels = len(glob.glob('../data/kernels/*/*/main.cu'))
errors = []

for idx, row in tqdm(runs.iterrows(), total=len(runs)):
    main_file_path = row['main_file']
    function_name = row['function']
    output_file_path = main_file_path.replace('main.cu', f'{function_name}.ll')
    if not path.exists(output_file_path):
        result = subprocess.run(compilation_command.format(main_file_path, output_file_path), shell=True)
        if result.returncode != 0:
            errors.append(main_file_path)

print(f"Compiled {num_kernels - len(errors)}/{num_kernels} nvidia kernels")
print(f"Errors at {errors}")

  0%|          | 0/20258 [00:00<?, ?it/s]

In file included from ../data/kernels/1/20/main.cu:10:
/usr/bin/../lib/gcc/x86_64-linux-gnu/9/../../../../include/c++/9/chrono:280:6: error: too few arguments provided to function-like macro invocation
        max() noexcept
            ^
../data/kernels/1/20/getRestricted.cu:2:9: note: macro 'max' defined here
#define max(a, b) ((a > b)?a:b)
        ^
In file included from ../data/kernels/1/20/main.cu:10:
/usr/bin/../lib/gcc/x86_64-linux-gnu/9/../../../../include/c++/9/chrono:280:2: error: declaration of constexpr static data member 'max' requires an initializer
        max() noexcept
        ^
/usr/bin/../lib/gcc/x86_64-linux-gnu/9/../../../../include/c++/9/chrono:280:5: error: expected ';' at end of declaration list
        max() noexcept
           ^
/usr/bin/../lib/gcc/x86_64-linux-gnu/9/../../../../include/c++/9/chrono:281:37: error: too few arguments provided to function-like macro invocation
        { return numeric_limits<_Rep>::max(); }
                                       

  1%|          | 201/20258 [00:48<2:22:04,  2.35it/s]